# Base Notebook
[1] [Deepchecks Installation](https://docs.deepchecks.com/stable/getting-started/installation.html#getting-started-installation)

[2] [Deepchecks NLP](https://docs.deepchecks.com/stable/nlp/index.html)

## 0. Import Library

for deepchecks

In [3]:
!pip install deepchecks --upgrade

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install "deepchecks[nlp]" --upgrade

Defaulting to user installation because normal site-packages is not writeable


for notebook

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

import re
#from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer

#from datasets import load_dataset

import warnings
warnings.filterwarnings(action='ignore')

In [6]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

# 나눔 폰트 경로를 직접 설정 
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = font_manager.FontProperties(fname=font_path)
plt.rcParams['font.family'] = fontprop.get_name()

In [7]:
from konlpy.tag import Kkma
kkma = Kkma()

## 1. Load Data

In [8]:
from datasets import load_dataset
data_cn = load_dataset("lbox/lbox_open", "casename_classification", download_mode="force_redownload")
data_cn

0000.parquet:   0%|          | 0.00/5.66M [00:00<?, ?B/s]

(…)e_classification/validation/0000.parquet:   0%|          | 0.00/739k [00:00<?, ?B/s]

(…)asename_classification/test/0000.parquet:   0%|          | 0.00/743k [00:00<?, ?B/s]

(…)sename_classification/test2/0000.parquet:   0%|          | 0.00/890k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test2 split:   0%|          | 0/1294 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'casetype', 'casename', 'facts'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'casetype', 'casename', 'facts'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'casetype', 'casename', 'facts'],
        num_rows: 1000
    })
    test2: Dataset({
        features: ['id', 'casetype', 'casename', 'facts'],
        num_rows: 1294
    })
})

In [9]:
# train + test 추출
train_data = data_cn['train']
test_data = data_cn['test']

train = pd.DataFrame(train_data)
test = pd.DataFrame(test_data)

train = train[['casetype', 'casename', 'facts']]
test = test[['casetype', 'casename', 'facts']]

In [10]:
train

,casetype,casename,facts
0,criminal,감염병의예방및관리에관한법률위반,피고인은 고양시 일산서구 B건물 C동에서 ‘D'을 실질적으로 운영하는 자이다.\n보...
1,criminal,감염병의예방및관리에관한법률위반,피고인은 성남시 중원구 B에서 ‘C'이라는 상호의 당구장을 운영하는 사람이다.\n중...
2,criminal,감염병의예방및관리에관한법률위반,"질병관리청장, 시·도지사 또는 시장·군수·구청장은 감염병을 예방하기 위하여나 감염병..."
3,criminal,감염병의예방및관리에관한법률위반,"질병관리청장, 시ㆍ도지사 또는 시장ㆍ군수ㆍ구청장은 감염병이 유행하면 감염병의 전파를..."
4,criminal,감염병의예방및관리에관한법률위반,"보건복지부장관, 시·도지사 또는 시장·군수·구청장은 감염병의 전파 방지 및 예방을 ..."
...,...,...,...
7995,criminal,횡령,범죄전력\n피고인은 2019. 12. 19. 서울중앙지방법원에서 도로교통법위반(음주...
7996,criminal,횡령,"기초사실\n피고인 B은 2018. 7.경 피고인 A에게 ‘신용도 문제가 있으니, 외..."
7997,criminal,횡령,범죄전력\n피고인 A은 2018. 4. 20. 대구지방법원 안동지원에서 사기죄 등으...
7998,criminal,횡령,피고인은 건설현장에서 목수로 일하는 사람으로 2012.경 지인의 소개로 피해자 B을...


In [11]:
test

,casetype,casename,facts
0,criminal,감염병의예방및관리에관한법률위반,"질병관리청장, 시·도지사 또는 시장·군수·구청장은 감염병이 유행하면 감염병의 전파를..."
1,criminal,감염병의예방및관리에관한법률위반,"질병관리청장, 시․도지사 또는 시장․군수․구청장은 감염병을 예방하기 위하여 감염병의..."
2,criminal,감염병의예방및관리에관한법률위반,보건복지부장관은 2020. 1. 8.경 코로나바이러스감염증-19(이하 ‘코로나-19...
3,criminal,감염병의예방및관리에관한법률위반,"질병관리청장, 시·도지사 또는 시장·군수·구청장은 감염병을 예방하기 위하여 흥행, ..."
4,criminal,감염병의예방및관리에관한법률위반,"피고인들은 수원시 영통구 C, D호에서 E 유흥주점을 공동으로 운영하는 사람들이다...."
...,...,...,...
995,criminal,횡령,피고인은 2019. 5.경 청주시 흥덕구 B에 있는 피해자 C이 운영하는 'D' 주...
996,criminal,횡령,"피고인은 2019. 1. 12.경 전남 목포시 B, 2층 C직업소개소에서 2019...."
997,criminal,횡령,피고인은 2018. 4. 24.경부터 2018. 7. 9.까지 당진시 B에 있는 C...
998,criminal,횡령,범죄전력\n피고인은 2019. 9. 6. 수원지방법원 평택지원에서 횡령죄로 징역 8...


In [12]:
print(train.shape, test.shape)
print()
print(train.info(), test.info())
print()
print(train.describe(), test.describe())

(8000, 3) (1000, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   casetype  8000 non-null   object
 1   casename  8000 non-null   object
 2   facts     8000 non-null   object
dtypes: object(3)
memory usage: 187.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   casetype  1000 non-null   object
 1   casename  1000 non-null   object
 2   facts     1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB
None None

        casetype          casename  \
count       8000              8000   
unique         2               100   
top     criminal  감염병의예방및관리에관한법률위반   
freq        5440                80   

                                                    facts  
count                                      

## 2. EDA

### Dist

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10)) 
plt.subplots_adjust(left=0.1, bottom=0.2, right=0.9, top=0.8, wspace=0.5, hspace=0.5)

# Train 데이터 casetype 분포
sns.countplot(y="casetype", data=train, order=train['casetype'].value_counts().index, ax=axes[0, 0])
axes[0, 0].set_xlabel('count', fontsize=8)
axes[0, 0].set_ylabel('casetype', fontsize=8)
axes[0, 0].set_title('Distribution of casetype (Train)')

# Train 데이터 casename 분포
sns.countplot(y="casename", data=train, order=train['casename'].value_counts().index, ax=axes[0, 1])
axes[0, 1].set_xlabel('count', fontsize=8)
axes[0, 1].set_ylabel('casename', fontsize=8)
axes[0, 1].set_title('Distribution of casename (Train)')

# test 데이터 casetype 분포
sns.countplot(y="casetype", data=test, order=test['casetype'].value_counts().index, ax=axes[1, 0])
axes[1, 0].set_xlabel('count', fontsize=8)
axes[1, 0].set_ylabel('casetype', fontsize=8)
axes[1, 0].set_title('Distribution of casetype (test)')

# test 데이터 casename 분포
sns.countplot(y="casename", data=test, order=test['casename'].value_counts().index, ax=axes[1, 1])
axes[1, 1].set_xlabel('count', fontsize=8)
axes[1, 1].set_ylabel('casename', fontsize=8)
axes[1, 1].set_title('Distribution of casename (test)')

plt.xticks(rotation=90)  # x축 글자 90도 회전
plt.show()


### len

In [ ]:
# train과 test 데이터셋의 'doc_len' 칼럼 생성
train['doc_len'] = train.facts.apply(lambda words: len(words.split()))
test['doc_len'] = test.facts.apply(lambda words: len(words.split()))

# plot
plt.figure(figsize=(10, 6))
sns.kdeplot(train['doc_len'], label='Train Document lengths', shade=True, color="blue")
sns.kdeplot(test['doc_len'], label='test Document lengths', shade=True, color="orange")
plt.axvline(x=train['doc_len'].mean(), color='blue', linestyle='--', label=f'Train mean: {train["doc_len"].mean():.0f}')
plt.axvline(x=test['doc_len'].mean(), color='orange', linestyle='--', label=f'test mean: {test["doc_len"].mean():.0f}')
plt.title("Document Lengths Distribution")
plt.xlabel("Document Length")
plt.ylabel("Density")
plt.legend()
plt.show()

# 가장 긴 문장과 가장 짧은 문장의 단어 수 출력
max_train_len = train['doc_len'].max()
min_train_len = train['doc_len'].min()
max_test_len = test['doc_len'].max()
min_test_len = test['doc_len'].min()

print(f"Train 데이터셋에서 가장 긴 문장: {max_train_len}개의 단어")
print(f"Train 데이터셋에서 가장 짧은 문장: {min_train_len}개의 단어")
print(f"test 데이터셋에서 가장 긴 문장: {max_test_len}개의 단어")
print(f"test 데이터셋에서 가장 짧은 문장: {min_test_len}개의 단어")

### Base WordCloud
토크나이저와 불용어 전처리를 모두 적용하지 않음

In [ ]:
def base_wordcloud(data, text_column_name):
    # 모든 텍스트를 하나의 문자열로 결합
    text = " ".join(data[text_column_name].dropna().tolist())
    
    # 워드 클라우드 생성
    wordcloud = WordCloud(
        font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf',  # 한글 폰트 경로
        width=800, 
        height=400, 
        background_color='white'
    ).generate(text)
    
    # 워드 클라우드 표시
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
print('train')
print (base_wordcloud(train, 'facts'))

print('test')
print( base_wordcloud(test, 'facts'))

## 3. [NLP Properties](https://docs.deepchecks.com/stable/nlp/usage_guides/nlp_properties.html)

In [14]:
from deepchecks.nlp import TextData

train_text = TextData(train['facts'])
train_text.calculate_builtin_properties()
train_text.properties

100%|██████████| 500/500 [00:04<00:00, 116.52it/s]


,Text Length,Average Word Length,Max Word Length,% Special Characters,% Punctuation,Language,Sentiment,Subjectivity,Average Words Per Sentence,Reading Ease,Lexical Density
0,482,3.455446,9,0.008299,0.066390,ko,NaN,NaN,6.867,NaN,0.84
1,225,3.652174,11,0.004444,0.048889,ko,NaN,NaN,5.750,NaN,0.89
2,363,3.513158,12,0.011019,0.055096,ko,NaN,NaN,6.333,NaN,0.87
3,346,3.360000,10,0.000000,0.057803,ko,NaN,NaN,5.357,NaN,0.80
4,382,3.301205,8,0.007853,0.068063,ko,NaN,NaN,4.941,NaN,0.83
...,...,...,...,...,...,...,...,...,...,...,...
7995,527,3.132231,11,0.005693,0.047438,ko,NaN,NaN,8.643,NaN,0.82
7996,416,3.168421,7,0.004808,0.050481,ko,NaN,NaN,8.727,NaN,0.84
7997,1105,2.934615,7,0.002715,0.072398,ko,NaN,NaN,8.387,NaN,0.68
7998,515,3.311927,8,0.015534,0.073786,ko,NaN,NaN,9.167,NaN,0.83


In [15]:
test_text = TextData(test['facts'])
test_text.calculate_builtin_properties()
test_text.properties

100%|██████████| 63/63 [00:00<00:00, 114.56it/s]


,Text Length,Average Word Length,Max Word Length,% Special Characters,% Punctuation,Language,Sentiment,Subjectivity,Average Words Per Sentence,Reading Ease,Lexical Density
0,423,3.311828,7,0.009456,0.052009,ko,NaN,NaN,6.200,NaN,0.84
1,473,3.179245,9,0.012685,0.061311,ko,NaN,NaN,7.067,NaN,0.79
2,488,3.375000,14,0.008197,0.067623,ko,NaN,NaN,5.778,NaN,0.83
3,593,2.874126,7,0.006745,0.065767,ko,NaN,NaN,7.526,NaN,0.72
4,411,3.344444,7,0.007299,0.051095,ko,NaN,NaN,6.923,NaN,0.82
...,...,...,...,...,...,...,...,...,...,...,...
995,320,3.191781,7,0.000000,0.046875,ko,NaN,NaN,9.125,NaN,0.90
996,336,3.131579,8,0.002976,0.065476,ko,NaN,NaN,9.500,NaN,0.79
997,335,3.012658,6,0.005970,0.050746,ko,NaN,NaN,11.286,NaN,0.84
998,880,3.177665,10,0.004545,0.061364,ko,NaN,NaN,7.880,NaN,0.64


## 4. NLP Embeddings

In [16]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np

# KLUE/RoBERTa 모델 및 토크나이저 로드
MODEL_NAME = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [18]:
from torch.utils.data import DataLoader, Dataset

# 1. 커스텀 데이터셋 정의
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        # `text` 또는 `facts` 열 자동 선택
        self.column_name = 'text' if 'text' in dataframe.columns else 'facts'
        self.texts = self.data[self.column_name]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts.iloc[idx]
    
# 2. 데이터셋 준비
train_dataset = CustomDataset(train)
test_dataset = CustomDataset(test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [20]:
def generate_embeddings(data_loader):
    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            inputs = tokenizer(list(batch), return_tensors="pt", padding=True, truncation=True, max_length=64).to("cuda")
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰의 임베딩 사용
            embeddings.append(cls_embeddings.cpu())
    return torch.cat(embeddings).numpy()

# 데이터 임베딩 생성
train_embeddings = generate_embeddings(train_loader)
test_embeddings = generate_embeddings(test_loader)

In [ ]:
# TextData 객체 생성 및 임베딩 추가
train_text.set_embeddings(train_embeddings)
test_text.set_embeddings(test_embeddings)

In [23]:
# Deepchecks 임베딩 드리프트 감지 수행
from deepchecks.nlp.checks import TextEmbeddingsDrift
check = TextEmbeddingsDrift()
result = check.run(train_text, test_text)
result.show() 

deepchecks - WARNING - Could not find model's classes, using the observed classes. In order to make sure the classes used by the model are inferred correctly, please use the model_classes argument


In [24]:
print("Drift Score:", result.value)


Drift Score: {'domain_classifier_auc': 0.48281666666666667, 'domain_classifier_drift_score': 0}


In [25]:
# 개별 샘플별 drift 확인
print(result.display)


[Figure({
    'data': [{'base': 0,
              'marker': {'color': '#01B8AA'},
              'offsetgroup': '0',
              'orientation': 'h',
              'showlegend': False,
              'type': 'bar',
              'x': [0],
              'y': [Drift Score]}],
    'layout': {'height': 200,
               'template': '...',
               'title': {'text': 'Drift Score - Multivariable'},
               'xaxis': {'dtick': 0.05,
                         'fixedrange': True,
                         'gridcolor': 'black',
                         'linecolor': 'black',
                         'range': [0, 0.4],
                         'showgrid': False,
                         'title': {'text': 'Drift score'}},
               'yaxis': {'autorange': True,
                         'color': 'black',
                         'fixedrange': True,
                         'rangemode': 'normal',
                         'showgrid': False,
                         'showline': False,
   